# Create points along a shp line and download laz tiles from Planetary computer

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import math
from shapely.geometry import Point
from shapely.geometry import shape
from shapely.ops import transform
import pystac_client
import stackstac
import planetary_computer
import pandas as pd
import os
import requests


# Function to create points along a line at specified intervals
def create_points_along_line(line, interval):
    length = line.length
    current_distance = 0
    points = []

    while current_distance < length:
        point = line.interpolate(current_distance)
        points.append(Point(point.x, point.y))
        current_distance += decimal_degrees

    return points

# Path to your shapefile
shapefile_path = "C:\\Path\\To\\SHP\\File\\TestPODbndry_BUCK_2.shp"

# Path to output laz files 
output_folder = "C:\\Path\\To\\Output\\LAZFile\\Location\\"

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Create a buffer around each geometry in the GeoDataFrame
buffered_gdf = gdf.buffer(0.01)

# Create a new GeoDataFrame from the buffered geometries
buffered_gdf = gpd.GeoDataFrame(geometry=buffered_gdf, crs=gdf.crs)

# Explore the buffered GeoDataFrame
buffered_gdf.explore()

meters_between_PTS=1000
xmin, ymin, xmax, ymax = buffered_gdf.total_bounds
latitude = (ymin + ymax) / 2
decimal_degrees = -meters_between_PTS / (111319.9 * math.cos(latitude))
print(decimal_degrees)

# Apply the function to each line in breached_pod and collect all points
all_points = []
for line in gdf.geometry:
    points = create_points_along_line(line, decimal_degrees)
    all_points.extend(points)

# Convert list of points to a GeoDataFrame
points_gdf = gpd.GeoDataFrame(geometry=all_points, crs=gdf.crs)

coordinates_list = [(point.x, point.y) for point in points_gdf.geometry]

# Construct the GeoJSON MultiPoint object
loc = {
    "type": "MultiPoint",
    "coordinates": coordinates_list
}

geom = shape(loc)
geom_buff = gpd.GeoSeries(geom.buffer(0.01), crs="EPSG:4326")
geom_buff.explore()


catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(
    collections=["3dep-lidar-copc"], intersects=geom.buffer(0.01)
)

ic = search.get_all_items()

df2 = gpd.GeoDataFrame.from_features(ic.to_dict(), crs="epsg:4326")
display(df2.head(5))
p1 = geom_buff.explore(color="orange")
p2 = df2.explore(m=p1, color="blue")
p2

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

for tile in ic:
    url = tile.assets["data"].href
    nm = tile.id + ".copc.laz"
    print("downloading " + nm)
    r = requests.get(url, allow_redirects=True)
    open(output_folder + "/" + nm, "wb").write(r.content)

print("DONE!")

# Save tile perimeters as .shp polygons

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
import pystac_client
import planetary_computer
import requests
import os

# Extract tile geometries from STAC items
tile_geometries = []

for tile in ic:
    tile_geom = shape(tile.geometry)
    tile_geometries.append(tile_geom)

# Create a GeoSeries from the list of tile geometries
tile_geometry_series = gpd.GeoSeries(tile_geometries, crs="epsg:4326")

# Create a GeoDataFrame from the GeoSeries
tile_gdf = gpd.GeoDataFrame(geometry=tile_geometry_series)

# Specify the path for saving the tile perimeter as a shapefile
tile_shapefile_path = "C:\\Path\\To\\Save\\Shp\\Poly\\Tile\\tile_perimeter.shp"

# Save the GeoDataFrame as a shapefile
tile_gdf.to_file(tile_shapefile_path)

print("Tile perimeter saved as a shapefile to", tile_shapefile_path)
